In [98]:
import csv

class Entry:
    def __init__(self, **kwargs):
        # credit | debit
        self.type = kwargs["type"]
        self.account = kwargs["account"]
        self.amount = kwargs["amount"]
    
    def __str__(self):
        return "{0} {1} {2}".format(self.type, self.account, f"{self.amount:,d}")

class JournalEntry:
    def __init__(self, **kwargs):
        self.date = kwargs["date"]
        self.entries = []
        self.memo = kwargs["memo"]
        
    def __str__(self):
        return "{0}\n{1}\n{2}".format(self.date, [str(e) for e in self.entries], self.memo)

class TAccountEntry:
    def __init__(self, **kwargs):
        self.account = kwargs["account"]
        self.date = kwargs["date"]
        # credit | debit
        self.type = kwargs["type"]
        self.amount = kwargs["amount"]
        self.memo = kwargs["memo"]
    
    def __str__(self):
        return "{0} {1} {2} {3} {4}".format(self.account, self.date, self.type, f"{self.amount:,d}", self.memo)

In [99]:
CSV_HEADER = 1

def add_entries(journalEntry, row):
    debit = row[1:4]
    credit = row[4:7]

    if debit[0] != "":
        journalEntry.entries.append(
            Entry(
                type="debit",
                account=debit[0].strip(),
                amount=int(debit[2])
            )
        )
    if credit[0] != "":
        journalEntry.entries.append(
            Entry(
                type="credit",
                account=credit[0].strip(),
                amount=int(credit[2])
            )
        )

def load_csv(file_path):
    journalEntries = []

    with open(file_path, newline='\n') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        # skip first line
        for i, row in enumerate(reader):
            if i <= CSV_HEADER:
                continue
                
            date = row[0].strip()
            
            # new journal entry
            if date != "":
                memo = (row[7] + ' ' + row[8]).strip()
                
                journalEntry = JournalEntry(date=date, memo=memo)  
                add_entries(journalEntry, row)
                journalEntries.append(journalEntry)
            else:
                journalEntry = journalEntries[-1]
                add_entries(journalEntry, row)
    
    return journalEntries

In [100]:
journalEntries = load_csv("/home/t4sk/Downloads/accounting-2020.csv")

In [101]:
def group_by_account(journalEntries):
    # account => entry[]
    grouped = {}
    
    for journalEntry in journalEntries:
        date = journalEntry.date
        memo = journalEntry.memo
        
        for entry in journalEntry.entries:
            account = entry.account
            
            if not account in grouped:
                grouped[account] = []
                
            grouped[account].append(
                TAccountEntry(
                    account=account,
                    date=date,
                    type=entry.type,
                    amount=entry.amount,
                    memo=memo
                )
            )

    return grouped
    

In [102]:
t_accounts = group_by_account(journalEntries)

In [115]:
def t_accounts_to_csv(t_accounts):
    rows = []
    
    for account, entries in t_accounts.items():
        # title
        rows.append([account])
        
        debit = 0
        credit = 0
        
        for entry in entries:
            # date, debit, credit, memo
            row = [
                entry.date,
                entry.amount if entry.type == "debit" else "",
                entry.amount if entry.type == "credit" else "",
                entry.memo
            ]
            
            if entry.type == "debit":
                debit += entry.amount
            elif entry.type == "credit":
                credit += entry.amount
            else:
                raise Exception(f'invalid entry type {entry.type}')

            rows.append(row)
        
        # append debit, credit total
        # skip, debit total, credit total
        rows.append(["", debit, credit])
    
    with open('t-accounts.csv', 'w', newline='\n') as file:
        writer= csv.writer(file, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for row in rows:
            writer.writerow(row)

t_accounts_to_csv(t_accounts)